<a href="https://colab.research.google.com/github/yodavo/IAI_vaccines/blob/main/data_wrangling/NPL_vaccines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Paquetes**

In [122]:
# Instalar paquetes no disponibles
!pip install fuzzywuzzy
#!pip install -U dataprep

# Para importar la data
import os

# Importar para manipulacion de datos
import numpy as np
import pandas as pd
import datetime as dt
from statistics import *

# Importar para Visualizacion 
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import warnings # para evitar warnings
warnings.filterwarnings('ignore')
import textwrap
from textwrap import wrap
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

# Importar Dependencias
%matplotlib inline
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


# **Revision de variables**

In [123]:
 from google.colab import drive
drive.mount('/content/drive')

# Entrando a la carpeta de trabajo
%cd '/content/drive/MyDrive/Colab Notebooks/Diplomado-UTEC/Modulo5/dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Diplomado-UTEC/Modulo5/dataset


In [124]:
# importar de VAERSData
data = pd.read_csv('pfizer_limpio.csv', index_col=0, encoding='latin-1')
print(f'VAER Pfizer-Hospitalized Data tiene\t: {data.shape[0]} observaciones y {data.shape[1]} variables.')

VAER Pfizer-Hospitalized Data tiene	: 10434 observaciones y 27 variables.


In [125]:
# cadenas a extraer texto
cadenas = ['HISTORY','SYMPTOM_TEXT', 'LAB_DATA', 'OTHER_MEDS', 'CUR_ILL', 'ALLERGIES', 'PRIOR_VAX']

# Creando columna INFO

In [126]:
# Cadenas de fillna
data.fillna({x:' ' for x in cadenas}, inplace=True)

# creamms una columan con toda la informacion textual
data['TEXT'] = data[cadenas].agg(" ".join, axis=1)

# eliminamos columnas que ya no aportan
data.drop(columns = cadenas, inplace=True)

In [127]:
# preprocesar texto
data.TEXT = data.TEXT.str.lower() # convertir en minuscula
data.TEXT = data.TEXT.str.replace('[^\w\s]',' ') # limpiar signos
data.TEXT = data.TEXT.str.rstrip() # limpiar espacios antes
data.TEXT = data.TEXT.str.lstrip() # limpiar especio despues

# Procesando INFO


In [128]:
# prueba para correr
# dataset = data.head(100).copy()
dataset = data.copy()

In [145]:
aesis = {'myocardic' : ['myocardic','acute myocardial infarction', 'Myocardial infarction', 
                      'Silent myocardial infarction', 'acute myocardial injury',
                      'acute myocardial ischemia'], 
         'anaphylactic' : ['anaphylactic ', 'Anaphylactic reaction', 'Anaphylactic shock', 
                         'Anaphylactoid reaction', 'Anaphylactoid shock'], 
         'coagulopathy' : ['coagulopathy','Acquired amegakaryocytic thrombocytopenia', 
                         'Amegakaryocytic thrombocytopenia', 'Axillary vein thrombosis', 
                         'Cavernous sinus thrombosis', 'Cerebral venous thrombosis', 
                         'Deep vein thrombosis', 'Disseminated intravascular coagulation', 
                         'Embolism venous', 'Hepatic vein thrombosis',
                         'Immune thrombocytopenia', 'Intracranial venous sinus thrombosis',
                         'Mesenteric vein thrombosis', 'Portal vein thrombosis', 
                         'Pulmonary embolism', 'Pulmonary thrombosis', 'Pulmonary venous thrombosis', 
                         'Severe fever with thrombocytopenia syndrome', 'Subclavian vein thrombosis', 
                         'Thrombocytopenia', 'Thrombocytopenic purpura', 
                         'Thrombotic thrombocytopenic purpura', 'Thrombosis', 
                         'Transverse sinus thrombosis', 'Vena cava embolism', 
                         'Vena cava thrombosis', 'Venous thrombosis'], 
         'covid19' : ['COVID-19', 'Asymptomatic COVID-19', 'COVID-19 pneumonia'],
         'death' : ['death', 'die'], 
         'guillian' : ['guillian', 'Guillian-Barre syndrome'], 
         'miller' : ['miller', 'Miller Fisher syndrome'], 
         'polyneurapathy' : ['polyneurapathy','Demyelinating polyneuropathy'],
         'kawasaki' : ['kawasaki', 'Kawasakis disease'],
         'multisystem' : ['multisystem','Multisystem inflammatory syndrome in children',
                          "Multisystem Inflammatory Syndrome in Adults (MIS-A)", 
                          "Atypical mycobacterium pericarditis", 
                          "Autoimmune myocarditis", "Autoimmune pericarditis", 
                          "Bacterial pericarditis"],
         'myocarditis' : ["Coxsackie myocarditis", "Coxsackie pericarditis", 
                          "Cytomegalovirus myocarditis", "Cytomegalovirus pericarditis", 
                          "Enterovirus myocarditis','Eosinophilic myocarditis", 
                          "Hypersensitivity myocarditis", "Immune mediated myocarditis", 
                          "Myocarditis", "Myocarditis bacterial", "Myocarditis helminthic", 
                          "Myocarditis infectious", "Myocarditis meningococcal", 
                          "Myocarditis mycotic", "Myocarditis post infection",
                          "Myocarditis septic", "Viral myocarditis"],
         'pericaditis' : ["Pericarditis","Pericarditis adhesive", "Pericarditis constrictive", 
                          "Pericarditis helminthic", "Pericarditis infective", 
                          "Pericarditis mycoplasma", "Pleuropericarditis", 
                          "Purulent pericarditis", "Viral pericarditis"],
         'narcolepsia' : ["Narcolepsy", "Cataplexy"],
         'abortion' : ["Aborted pregnancy", "Abortion complete"," Abortion early", 
                       "Abortion incomplete", "Abortion late", "Abortion missed", 
                       "Abortion spontaneous", "Abortion spontaneous complete", 
                       "Abortion spontaneous incomplete", "Abortion threatened", 
                       "Congenital anomaly", "Drug exposure during pregnancy", 
                       "Exposure during pregnancy", "Foetal death Maternal",
                       "exposure during pregnancy" ,"Stillbirth"],
         'epilepsy' : ["Convulsions  Atonic seizures", "Atypical benign partial epilepsy",
                       "Autonomic seizure", "Clonic convulsion", "Complex partial seizures", 
                       "Convulsion in childhood", "Convulsion", "Convulsions local", 
                       "Epilepsy" "Epileptic encephalopathy" "Febrile convulsion", 
                       "Febrile infection-related epilepsy syndrome", 
                       "Generalised non-convulsive epilepsy", "Generalised onset non-motor seizure", 
                       "Generalised tonic-clonic seizure", "Grand mal convulsion", 
                       "Idiopathic generalised epilepsy", "Myoclonic epilepsy", 
                       "Neonatal seizure", "Partial seizures with secondary generalisation", 
                       "Partial seizures", "Petit mal epilepsy", "Seizure anoxic", 
                       "Seizure cluster", "Seizure like phenomena", "Seizure", 
                       "Simple partial seizures", "Status epilepticus", 
                       "Temporal lobe epilepsy","Tonic clonic movements", 
                       "Tonic convulsion", "Tonic posturing", "Basal ganglia stroke", 
                       "Brain stem stroke", "Cerebellar stroke", "Cerebrovascular accident", 
                       "Embolic stroke","Haemorrhagic stroke", "Haemorrhagic transformation stroke", 
                       "Ischaemic stroke", "Lacunar stroke", "Perinatal stroke", 
                       "Spinal stroke", "Thrombotic stroke", "Vertebrobasilar stroke", 
                       "Transverse myelitis"],
         'adrs' : ["Acute Respiratory Distress Syndrome"],
         'autoimmuned' : ["Autoimmune disorders NEC", "Blood autoimmune disorders", 
                          "Endocrine autoimmune disorders", "Hepatic autoimmune disorders", 
                          "Lupus erythematosus and associated conditions", 
                          "Muscular autoimmune disorders", "Nervous system autoimmune disorders", 
                          "Rheumatoid arthritis and associated conditions", 
                          "Scleroderma and associated disorders", "Skin autoimmune disorders NEC"],
         'adem' : ["encephalomyelitis", "Acute disseminated encephalomyelitis", "Encephalomyelitis", 
                   "Leukoencephalomyelitis", "Noninfective encephalomyeliti"],
         'mesclerosis' : ["Multiple sclerosis", "Multiple sclerosis relapse", 
                          "Primary progressive multiple sclerosis", "Progressive multiple sclerosis", 
                          "Progressive relapsing multiple sclerosis", "Relapsing multiple sclerosis", 
                          "Relapsing-remitting multiple sclerosis", 
                          "Secondary progressive multiple sclerosis", "Tumefactive multiple sclerosis"],
         'Oneuritis' : ["Optic neuritis", "neuritis"],
         'CIDP' : ["Chronic inflammatory demyelinating polyneuropathy"],
         'encephalitis' : ["Encephalitis"],
         'meningoencephalitis' : ["Meningoencephalitis viral"],
         'meningitis' : ["Meningitis", "Meningitis aseptic", "Meningitis viral"],
         'encephalopathy' : ["Encephalopathy", "Leukoencephalopathy"],
         'ataxia' : ["Ataxia", "Cerebellar ataxia", "Cerebral ataxia"],
         'noanaphylactic' : ["Allergic reaction to excipient", "Allergy to vaccine", 
                             "Allergic bronchitis", "Allergic colitis", "Allergic cough", 
                             "Allergic cystitis", "Allergic gastroenteritis", 
                             "Allergic hepatitis", "Allergic keratitis", 
                             "Allergic pharyngitis", "Allergic reaction to excipient", 
                             "Allergic respiratory disease", "Allergic respiratory symptom", 
                             "Allergic sinusitis", "Conjunctivitis allergic",
                             "Dermatitis allergic", "Encephalitis allergic", 
                             "Encephalopathy allergic", "Laryngitis allergic", 
                             "Nephritis allergic", "Pruritus allergic", "Rhinitis allergic"]
         }

In [146]:
# Funcion de cambio para usar el diccionario de eventos aesi con cada observacion del dataframe
def aesisDetect(row, aesi):
  from fuzzywuzzy import fuzz
  sinonimos = aesis[aesi]
  contador = 0
  for sinonimo in sinonimos:
    if fuzz.partial_ratio(row,sinonimo.lower()) > 85 :
      #print(f'{sinonimo.lower()} : en : {row}')
      contador += 1
  return contador

In [147]:
# Procesamiento de lenguaje natural
for aesi in aesis.keys():
  dataset[aesi] = dataset.TEXT.apply(lambda x: aesisDetect(x, aesi))

# Visualizando los valores
for aesi in aesis.keys():
  print(dataset[aesi].value_counts())

0    10203
1      201
4       14
3        7
2        7
5        2
Name: myocardic, dtype: int64
0    10411
3       12
2        8
1        3
Name: anaphylactic, dtype: int64
0    10167
1      215
2       42
3       10
Name: coagulopathy, dtype: int64
0    7078
1    3303
2      53
Name: covid19, dtype: int64
0    10347
1       87
Name: death, dtype: int64
0    10384
2       26
1       24
Name: guillian, dtype: int64
0    10433
1        1
Name: miller, dtype: int64
0    10430
1        3
2        1
Name: polyneurapathy, dtype: int64
0    10430
2        4
Name: kawasaki, dtype: int64
0    10428
3        3
1        3
Name: multisystem, dtype: int64
0     10232
1       196
2         3
14        2
3         1
Name: myocarditis, dtype: int64
0    10360
1       71
9        3
Name: pericaditis, dtype: int64
0    10432
2        1
1        1
Name: narcolepsia, dtype: int64
0    10428
1        5
3        1
Name: abortion, dtype: int64
0     9968
1      447
2       15
12       2
13       1
4        1

In [148]:
# Reemplazando valores numericos a binario (tipo onehotencoding)
for i in list(aesis.keys()):
  dataset[i] = np.where(dataset[i] > 0, 1, dataset[i])

# Visualizando los valores
for aesi in aesis.keys():
  print(dataset[aesi].value_counts())

0    10203
1      231
Name: myocardic, dtype: int64
0    10411
1       23
Name: anaphylactic, dtype: int64
0    10167
1      267
Name: coagulopathy, dtype: int64
0    7078
1    3356
Name: covid19, dtype: int64
0    10347
1       87
Name: death, dtype: int64
0    10384
1       50
Name: guillian, dtype: int64
0    10433
1        1
Name: miller, dtype: int64
0    10430
1        4
Name: polyneurapathy, dtype: int64
0    10430
1        4
Name: kawasaki, dtype: int64
0    10428
1        6
Name: multisystem, dtype: int64
0    10232
1      202
Name: myocarditis, dtype: int64
0    10360
1       74
Name: pericaditis, dtype: int64
0    10432
1        2
Name: narcolepsia, dtype: int64
0    10428
1        6
Name: abortion, dtype: int64
0    9968
1     466
Name: epilepsy, dtype: int64
0    10433
1        1
Name: adrs, dtype: int64
0    10433
1        1
Name: autoimmuned, dtype: int64
0    10434
Name: adem, dtype: int64
0    10403
1       31
Name: mesclerosis, dtype: int64
0    10432
1        2
Name:

In [149]:
# creamos una columna con toda la informacion textual
dataset['decision'] = dataset[list(aesis.keys())].agg(np.sum, axis=1)

# Visualizando los valores
dataset.decision.value_counts()

0    6090
1    3866
2     434
3      44
Name: decision, dtype: int64

In [150]:
dataset.to_csv('/content/sample_data/dataset_aesi.csv')

In [ ]:
---

In [152]:
## Ultima revision
dataset.loc[dataset['decision'] > 0].shape

(4344, 49)

# **Otras**

In [ ]:
sintomas_objetivo = ['vaccine-associated enhanced disease', 'multisystem inflammatory syndrome in children', 'acute respiratory distress syndrome',
                     'acute cardiovascular injury', 'microangiopathy', 'heart failure', 'stress cardiomyopathy', 'coronary artery disease arrhythmia',
                     'myocarditis', 'Coagulation disorder', 'thromboembolism', 'haemorrhage', 'Acute kidney injury', 'Generalized convulsion', 
                     'Acute liver injury', 'anosmia','ageusia', 'Chilblain', 'lesion', 'Single organ cutaneous vasculitis', 'Erythema multiforme', 
                     'Anaphylaxis', 'Acute aseptic arthritis', 'Meningoencephalitis', 'Acute disseminated encephalomyelitis', 'Thrombocytopenia'
                     ]
sintomas_objetivo = list(map(str.lower,sintomas_objetivo))
sintomas_objetivo.sort()

# Notas previas

POSIBLES OBJETIVOS DEL TRABAJO DE INVESTIGACION:

1. Personalizacion de las vacunas Moderna, Pfizer  y Johnson & Johnson pra reducir efectos adversos en perfiles especificos en pacientes. Este objetivo requeriria de un dataset de control con pacientes que no presentaron reacciones adversas a las vacunas.

- Target: tipo de vacuna
- Features: Datos de los pacientes (alergias, comorbolidades, edad, sexo, 
enfermedades cronicas y actuales (historia clinica), vacunas previas, sintomas post aplicacion, hospitalizacion y dias de hospitilizacion, discapacidades...)

2. Series de tiempo frente antes y despues de la fecha de vacunacion.

3. Predecir el sexo y/o rango de edad basado en los sintomas presetnados post aplicacion de las vacunas A. B.

4. Predecir si el paciente requiere hospitalizacion basado en los sintomas que presenta al inocularse la vacuna por clusters (no supervisado).

5. Predecir que sintomas puede presentar el paciente en base a su sexo y la vacuna inoculada.